In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
# Paramètres
input_shape = (224, 224, 3)
num_classes = 30  # Remplacez par le nombre de classes de votre tâche
batch_size = 32
epochs = 50

In [3]:
data_dir = "Segmented Medicinal Leaf Images"

In [4]:
#augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [10]:
# Chargez les données d'entraînement et de validation en utilisant le générateur
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size = input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Utilisez la sous-partie d'entraînement
)


Found 1479 images belonging to 30 classes.


In [11]:
test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Utilisez la sous-partie de validation
)

Found 356 images belonging to 30 classes.


In [12]:
# Chargement du modèle pré-entraîné VGG16 avec les poids ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)


58889256/58889256 [==============================] - 54s 1us/step


In [13]:
# Geler les couches du modèle de base (sauf les dernières couches de classification)
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [14]:
# Création du modèle personnalisé pour la tâche de classification
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [15]:
# Création du modèle final
model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
# Utilisation de l'arrêt précoce
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


In [20]:
# Entraînement du modèle
model.fit(
    train_generator,
    epochs=epochs,
    callbacks=[early_stopping],
    validation_data=test_generator  
)

Epoch 1/50
47/47 [==============================] - 839s 18s/step - loss: 2.6236 - accuracy: 0.2684 - val_loss: 1.5463 - val_accuracy: 0.5169
Epoch 2/50
47/47 [==============================] - 797s 17s/step - loss: 1.2468 - accuracy: 0.6126 - val_loss: 0.9280 - val_accuracy: 0.7219
Epoch 3/50
47/47 [==============================] - 799s 17s/step - loss: 0.7263 - accuracy: 0.7681 - val_loss: 0.6165 - val_accuracy: 0.8118
Epoch 4/50
47/47 [==============================] - 899s 19s/step - loss: 0.5378 - accuracy: 0.8208 - val_loss: 0.5706 - val_accuracy: 0.8287
Epoch 5/50
47/47 [==============================] - 885s 19s/step - loss: 0.3468 - accuracy: 0.8857 - val_loss: 0.3888 - val_accuracy: 0.8848
Epoch 6/50
47/47 [==============================] - 939s 20s/step - loss: 0.3116 - accuracy: 0.9013 - val_loss: 0.4078 - val_accuracy: 0.8567
Epoch 7/50
47/47 [==============================] - 941s 20s/step - loss: 0.2605 - accuracy: 0.9182 - val_loss: 0.4286 - val_accuracy: 0.8483
Epoch 

In [21]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(test_generator)  # Remplacez par votre générateur de test
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

12/12 [==============================] - 129s 11s/step - loss: 0.2476 - accuracy: 0.9354
Test accuracy: 93.54%


In [22]:
# Entraînement du modèle
model.fit(
    train_generator,
    epochs=18,
    callbacks=[early_stopping],
    validation_data=test_generator  # Remplacez par votre générateur de validation
)

Epoch 1/18
47/47 [==============================] - 802s 17s/step - loss: 0.0467 - accuracy: 0.9858 - val_loss: 0.2779 - val_accuracy: 0.9354
Epoch 2/18
47/47 [==============================] - 798s 17s/step - loss: 0.0579 - accuracy: 0.9804 - val_loss: 0.4026 - val_accuracy: 0.9017
Epoch 3/18
47/47 [==============================] - 799s 17s/step - loss: 0.0758 - accuracy: 0.9743 - val_loss: 0.5777 - val_accuracy: 0.8652
Epoch 4/18
47/47 [==============================] - 791s 17s/step - loss: 0.1895 - accuracy: 0.9520 - val_loss: 0.5550 - val_accuracy: 0.8848
Epoch 5/18
47/47 [==============================] - 789s 17s/step - loss: 0.0857 - accuracy: 0.9763 - val_loss: 0.3782 - val_accuracy: 0.9101
Epoch 6/18
47/47 [==============================] - 798s 17s/step - loss: 0.0421 - accuracy: 0.9858 - val_loss: 0.3627 - val_accuracy: 0.9073
Epoch 7/18
47/47 [==============================] - 792s 17s/step - loss: 0.0542 - accuracy: 0.9851 - val_loss: 0.3028 - val_accuracy: 0.9270
Epoch 

In [23]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(test_generator)  # Remplacez par votre générateur de test
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

12/12 [==============================] - 130s 11s/step - loss: 0.2116 - accuracy: 0.9494
Test accuracy: 94.94%
